In [3]:
import pandas as pd
import numpy as np

In [4]:
df_pred = pd.read_csv('./VertexAI/PaLM_test_result1.txt', delimiter='|',header=None, names=['Sentences', 'Present', 'Code Word','Target Word'])
df_actual = pd.read_csv('./BigQuery/Files/CombinesDS_2.txt', delimiter='|')

In [5]:
df_pred['Present'] = df_pred['Present'].str.lower()
df_pred.head(5)

,Sentences,Present,Code Word,Target Word
0,"all up my nose already haha, was a very very f...",yes,crystal,Methamphetamine
1,nah he extended his 'christmas' sale because h...,yes,crystal,Methamphetamine
2,boy you really saved us all with one. almost w...,no,NA,NA
3,ask yourself if you would eat another apple,no,NA,NA
4,the consistency was very similar to baby powde...,yes,#4,Heroin


In [6]:
df_actual.head()

,Sentences,Present,Code Word,Target Word
0,"all up my nose already haha, was a very very f...",yes,crystal,methamphetamine
1,nah he extended his 'christmas' sale because h...,yes,crystal,methamphetamine
2,boy you really saved us all with one. almost w...,no,NaN,NaN
3,ask yourself if you would eat another apple,no,NaN,NaN
4,the consistency was very similar to baby powde...,yes,baby powder,cocaine


In [7]:
df_actual = df_actual.iloc[:103]
df_actual.shape

(103, 4)

In [8]:
actual_val = df_actual['Present'].astype(str).replace({'yes': 1, 'no': 0})
predicted_val = df_pred['Present'].str.strip().astype(str).replace({'yes': 1, 'no': 0})

In [9]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

cm= confusion_matrix(actual_val, predicted_val)
print("Confusion Matrix:\n", cm)

report = classification_report(actual_val, predicted_val)
print("Classification Report:\n", report)

print(f"Accuracy is {accuracy_score(actual_val, predicted_val)}")

Confusion Matrix:
 [[54  4]
 [ 8 37]]
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.93      0.90        58
           1       0.90      0.82      0.86        45

    accuracy                           0.88       103
   macro avg       0.89      0.88      0.88       103
weighted avg       0.88      0.88      0.88       103

Accuracy is 0.883495145631068


In [10]:
df_pred2 = df_pred
df_pred2 = df_pred2.rename(columns={'Present': 'Pred_Present', 'Code Word': 'Pred_Code_Word','Target Word':'Pred_target_word'})
df_pred2.head()

,Sentences,Pred_Present,Pred_Code_Word,Pred_target_word
0,"all up my nose already haha, was a very very f...",yes,crystal,Methamphetamine
1,nah he extended his 'christmas' sale because h...,yes,crystal,Methamphetamine
2,boy you really saved us all with one. almost w...,no,NA,NA
3,ask yourself if you would eat another apple,no,NA,NA
4,the consistency was very similar to baby powde...,yes,#4,Heroin


In [11]:
df_pred2['Sentences'] = df_pred2['Sentences'].str.strip()
df_actual['Sentences'] = df_actual['Sentences'].str.strip()

In [12]:
matching = df_pred2['Sentences'].equals(df_actual['Sentences'])
if matching:
    print("Column A in both DataFrames is matching")
else:
    print("Column A in both DataFrames is NOT matching")

Column A in both DataFrames is matching


In [13]:
combined = pd.merge(df_actual, df_pred2, on='Sentences')

In [14]:
combined[combined.duplicated()]

,Sentences,Present,Code Word,Target Word,Pred_Present,Pred_Code_Word,Pred_target_word


In [16]:
#combined.head()
combined.to_csv('Result_file_Bart1.txt', sep='|',index=False)